# MPS canonical forms 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import numpy as np

import copy 

import myMPSstuff as mps
import myMPOstuff as mpo

import myIsingMPO as isi
import myMPOMPS as mpomps

from myUtils import sncon as ncon
from myUtils import real_close as rc
from myUtils import checkIdMatrix

import myEnvironmentsMPO as envs



In [3]:

test2 = mps.myMPS(mps.randMPS(chi=100))
print(f"Entropies=  {test2.getEntropies()}")

print(type(test2))
print(type(test2.MPS))


Entropies=  [0.0, 1.3222656501872186e-05, 8.096534644701398e-07, 7.8723506315777e-07, 1.1737047584651014e-07, 6.438246046194521e-08, 2.569399823963767e-07, 9.721000449553574e-08, 1.199602076941477e-06, 9.861370787303927e-06, 6.661338147750936e-16]
<class 'myMPSstuff.myMPS'>
<class 'list'>


In [4]:

testMPO = mpo.myMPO(isi.IsingMPO(LL=10, J = 1, g = 0.4))


In [5]:

#mpstest = np.random.rand(4,7,3,7)

test = mps.myMPS()

print(test.chis)
print([np.shape(s) for s in test.MPS])

print(test.getNormSlow())
print(test.getNorm())



[1, 20, 20, 20, 20, 20, 20, 1]
[(1, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 20, 2), (20, 1, 2)]
(71634487.91145544+1.0906059675692416e-10j)
71634487.91145544


In [6]:
test = mps.myMPS(mps.plusState(10))

print(test.chis)
print([np.shape(s) for s in test.MPS])

# the state should be normalized already 
print(test.getNormSlow())
print(test.getNorm())

test.bringCan()
print(test.getEntropies())  # should be zero for a prod state



[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[(1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2), (1, 1, 2)]
0.9999999999999989
0.9999999999999992
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [7]:
test = mps.myMPS()
print(test.getNormSlow())

print(test.getNorm())
test.bringCan()

(71634487.91145544+1.0906059675692416e-10j)
71634487.91145544


'L'

In [8]:


test3 = mps.myMPS(mps.randMPS(LL=20, chi=200))
print(f"Entropies=  {test3.getEntropies()}")

testMPO = mpo.myMPO()




Entropies=  [0.0, 1.2630591034956043e-06, 3.138693711496794e-07, 2.7647643933426194e-08, 3.043902441702274e-08, 5.710164457502063e-08, 4.576140498533507e-08, 1.1899803406489368e-07, 1.506169679591301e-07, 1.4188084037992554e-07, 2.2102182897632132e-09, 1.4947375748536123e-07, 2.5048302497119856e-08, 3.8859953336183724e-08, 6.540629119289407e-09, 1.301045193908247e-07, 2.9468698573895984e-08, 3.452539858962834e-09, 1.5205943654348293e-07, 8.31889558261259e-06, -4.4408920985006364e-15]


## Check that we're actually in proper L-R canon forms by contracting a bit and see if we recover identity matrices and normalizations 

In [9]:
mymps3 = mps.myMPS(mps.randMPS(40,chi=300))

mymps3.bringCan(epsTrunc=1e-10,epsNorm=1e-14, chiMax=200)
print(mymps3.curr_form)
# print(mymps3.SV)
# print(mymps3.SVinv)
print(mymps3.chis)

# Now set to R 
mymps3.set_form(mode='R')
print(mymps3.curr_form)
# So now the MPS should be normalized and in  RIGHT canonical form 
# Check by contracting some B's 

print(f"Norm={mymps3.getNorm()}")

[ checkIdMatrix(ncon([m,np.conj(m)],[[-1,1,2],[-2,1,2]])) for m in mymps3.MPS ]


mymps3.set_form(mode='L')
print(mymps3.curr_form)
# print(mymps3.SV)
# print(mymps3.SVinv)


# So now the MPS should be normalized and in  LEFT canonical form 
# Check by contracting some A's 

print(f"Norm={mymps3.getNorm()}")

[ checkIdMatrix(ncon([m,np.conj(m)],[[1,-1,2],[1,-2,2]])) for m in mymps3.MPS ]

# # Now set to R again 
# mymps3.set_form(mode='R')
# print(mymps3.curr_form)
# # So now the MPS should be normalized and in  RIGHT canonical form 
# # Check by contracting some B's 

# print(f"Norm={mymps3.getNorm()}")

# [ checkIdMatrix(ncon([m,np.conj(m)],[[-1,1,2],[-2,1,2]])) for m in mymps3.MPS ]



L
[1, 2, 4, 8, 4, 4, 5, 4, 4, 4, 5, 5, 4, 5, 4, 4, 4, 5, 5, 4, 5, 4, 5, 4, 5, 4, 4, 5, 4, 4, 5, 5, 4, 5, 4, 4, 5, 7, 4, 2, 1]
R
Norm=1.0000000000000036
identity, size = 1
identity, size = 2
identity, size = 4
identity, size = 8
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 5
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 4
identity, size = 5
identity, size = 7
identity, size = 4
identity, size = 2
L
Norm=1.0000000000000036
identity, size = 2
identity, size = 4
identity, size = 8
ident

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [10]:
m =mymps3.MPS[8]
np.real_if_close(np.around(ncon([m,np.conj(m)],[[1,-1,2],[1,-2,2]]),12))

print(mymps3.SV[7][0])
print(np.linalg.norm((mymps3.SV[7])))
print(mymps3.SV[8])
print(np.linalg.norm((mymps3.SV[8])))
print(mymps3.SVinv[8])
print(np.linalg.norm((mymps3.SVinv[8])))

0.9999999999188647
1.0000000000000004
[9.99999999e-01 3.53316180e-05 3.80894081e-08 2.18096358e-08]
1.0000000000000007
[1.00000000e+00 2.83032608e+04 2.62540179e+07 4.58512930e+07]
52835739.08505169


## Try computing an expectation value, first the "hard" way (ncon the whole thing), then using the right-canonical expression

In [11]:
# Try to do it a bit better, define separately psi and psi*
# and compute the norm 

testL = 12
testd = 2
testChi = 8

mps1 = mps.randMPS(testL,chi=testChi, d=testd)

mypsi = mps.myMPS(mps1)
#mypsic = mps.myMPS([np.conj(m) for m in mps1])

# make two copies for later messing around 
mypsic1 = copy.deepcopy(mypsi)
mypsic1conj = mps.myMPS([np.conj(m) for m in mypsic1.MPS])

mypsic2 = copy.deepcopy(mypsi)


# Now define the operator to exp-value,
# say we wanna compute <sigma_x> 
op1 = np.array([[0,1.],[1.,0]])

# at site r = 3 

print("Exp value the hard way: ")

insertjj = 3 
tryind = [[1,3,2]]
tryind.append([1,4,2])

for jj in range(1,testL):
    tj = 3*jj
    if jj == insertjj: # horrible hack 
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+2,9999])
        tryind.append([tj+1,tj+4,9999])
    else:
        tryind.append([tj,tj+3,tj+2])
        tryind.append([tj+1,tj+4,tj+2])

tryind[-2][1] = tryind[-1][1]

opList = []
for jj, m in enumerate(mypsi.MPS):
    if jj == insertjj:
        opList.append(m)
        opList.append(op1)
        opList.append(m.conj())
    else:
        opList.append(m)
        opList.append(m.conj())

ev = ncon(opList,tryind)

#print(tryind )
print(np.real_if_close(ev/mypsi.getNormSlow() ))
#print(np.real_if_close(ev/mypsi.getNorm() ))

print("Second way: apply the op to the relevant site, then compute the overlap with itself")
mypsic1.MPS[3] = ncon([op1,mypsic1.MPS[3]],[[-3,1],[-1,-2,1]])
expvaloverlap = mypsic1.overlap(mypsic1conj) 
print(np.real_if_close(expvaloverlap/mypsic1.getNormSlow() ))

print(" Now compute it with the canonical form ")

mypsic2.set_form('R')

conTen = [np.diag(mypsic2.SV[3]),np.diag(mypsic2.SV[3]),mypsic2.MPS[3],np.conj(mypsic2.MPS[3]),op1]
conIdx = [[1,2],[1,3],[3,5,4],[2,5,6],[4,6]]

[print(np.shape(elem)) for elem in conTen]
print(len(conTen),len(conIdx))

print(np.real_if_close(ncon(conTen,conIdx)))


Exp value the hard way: 
(8553445332.681737+3.9513141676414005e-07j)
Second way: apply the op to the relevant site, then compute the overlap with itself
(8553445332.681738-7.425130922421363e-09j)
 Now compute it with the canonical form 
(8, 8)
(8, 8)
(8, 8, 2)
(8, 8, 2)
(2, 2)
5 5
0.99616805993749


# build and test environments

In [12]:
o = mpo.myMPO(isi.IsingMPO(LL = 20))

psi = mps.myMPS(mps.randMPS(LL=o.LL))

In [13]:
le, re = envs.build_environments(psi, o)

print(mpomps.expValMPO(psi, o))

for j in range(2,psi.LL):
    #print(np.shape(le[j]), np.shape(re[j]), np.shape(re[j+1]))
    expval = np.real_if_close(ncon([le[j],re[j]],[[1,2,3],[1,2,3]]))
    print(j, expval)


-8.076737197963357
2 -8.076737197963357
3 -8.076737197963357
4 -8.076737197963359
5 -8.076737197963356
6 -8.076737197963356
7 -8.076737197963354
8 -8.076737197963352
9 -8.076737197963356
10 -8.076737197963354
11 -8.076737197963356
12 -8.076737197963359
13 -8.076737197963357
14 -8.076737197963357
15 -8.076737197963359
16 -8.076737197963359
17 -8.076737197963359
18 -8.076737197963359
19 -8.076737197963359


In [14]:
print(rc(le[-1])) # should be a number

[[[1.]]]


In [15]:
oN = mpo.myMPO(mpo.trivial_sx_MPO(LL = 20))

In [16]:
import myPowerMethod as pow

pow.power_method(oN, 0, 50, 100, 0, True)

Returning energies as expectation value of the evolution MPO


6, chi/max 10/10:   5%|▌         | 5/100 [00:00<00:10,  8.87it/s]


Looks converged after 6 steps
max chi reached(/max): 10/50, final dSmid = 6.664319703725274e-16
Returning *ALL* entropies


(<myMPSstuff.myMPS at 0x1158a6880>,
 [2, 4, 6],
 [[0.0,
   0.0012923193557686386,
   0.00018304241691212257,
   0.00015039134636785117,
   0.0001804442023113296,
   7.628586371412643e-05,
   0.00023092188587085652,
   0.00047859720477673673,
   0.0002298832709361652,
   4.075991965601523e-05,
   0.0002413294481675578,
   2.2711165214424583e-05,
   2.6001766912920428e-05,
   0.00010043481335782435,
   0.00018773582290111549,
   1.1190068617164372e-05,
   0.00018779372365205908,
   0.0010473960869890617,
   0.0003479142995589798,
   0.0002624926402069189,
   -8.881784197001256e-16],
  [0.0,
   0.0012923193557686388,
   0.00018304241691079025,
   0.000150391346366741,
   0.00018044420230977598,
   7.628586371257232e-05,
   0.00023092188586930316,
   0.0004785972047756272,
   0.00022988327093483328,
   4.075991965446085e-05,
   0.00024132944816622568,
   2.2711165213536492e-05,
   2.6001766912032293e-05,
   0.00010043481335782442,
   0.0001877358229011153,
   1.1190068616720278e-05,
   0.0